In [3]:
import sys
import os
import numpy as np
import time
from imp import reload

In [4]:
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE


# Test 1: DAQ Controllers


In [3]:
from L1 import DAQControllers

## National Instruments DAQ

Tests:
1. Initialize the DAQ
2. Set an analog output Voltage
3. Read an analog input voltage
4. Set an digitalouput voltage

In [4]:
ni = DAQControllers.NiDaq()

def print_data(data,time,channels,*args):
    print(f"Total samples: {len(data[0])}, average: {np.mean(data)} V ")
    
ni.add_callback(print_data, ['ai0'], "wave",())
print("Initialize complete...")

Initialize complete...


In [5]:
ni.add_analog_output('ao0')
ni.set_channel_voltage('ao0',0.05)
ni.start_voltage()
print("Analog output voltage set, measure channel ao0. \nMeter should read 50 mV")

Analog output voltage set, measure channel ao0. 
Meter should read 50 mV


In [6]:
ni.add_analog_input('ai0')
ni.set_sampling_frequency(1000)
ni.start_measurement()
time.sleep(.1)
ni.stop_measurement()
print("\nThere should be 100 samples read in 0.1 s \n")

Total samples: 100, average: -5.310797896931989 V 
There should be 100 samples read in 0.1 s 




In [7]:
ni.add_do_channel('p0.0')
ni.set_do_channel('p0.0', True)
ni.update_do_channels()
print("Digital output on Port0/line0 should read digital High")

Digital output on Port0/line0 should read digital High


# Test 2: Linear Motion Utilities

tests:
1. Home to Zero position
2. Read and set position
3. Stop movement

### Thorlab Kinesis Motor

In [256]:
from L1 import Controllers
from L2 import ZControl
reload(ZControl)
reload(Controllers)

<module 'L1.Controllers' from 'C:\\Users\\NikonTE300CE\\Desktop\\Barracuda\\AutomatedCE\\L1\\Controllers.py'>

In [104]:
ser_num = '49125264'
z = ZControl.KinesisZ(ser_num,'zstage')
z.startup()
print("Thorlabs Initialized...")


DeviceNotReadyException: Device is not connected
   at Thorlabs.MotionControl.DeviceManagerCLI.ThorlabsGenericCoreDeviceCLI.VerifyDeviceConnected(Int32 functionDepth)
   at Thorlabs.MotionControl.GenericMotorCLI.GenericMotorCLI.Connect(String serialNo)

In [257]:
ctl = Controllers.ArduinoController("COM8")
ctl.open()
z = ZControl.ArduinoZ(ctl, 'zstage')
z.startup()

TESTING12345

[b'\x00\x00TESTING12345\r\n']
0 59.75 60
M0L+29.750

[b'M0L+29.750\r\n', b'OK\r\n']
TESTING12345

[b'TESTING12345\r\n']
0 59.75 60
M0L+29.750

[b'M0L+29.750\r\n', b'OK\r\n']


In [164]:
ctl._serial.write("M0L?\n".encode())
a = ctl._serial.read_until('\r\n'.encode())
a = ctl._serial.read_until('\r\n'.encode())
a

b'M0L?\r\n'

In [224]:
ctl._serial.write("M0L+10.200\n".encode())
time.sleep(0.2)
a = ctl._serial.read_all()
print(a)
z.wait_for_move()

b'M0L+10.200\r\nOK\r\n'
M0L?

[b'M0L?\r\n', b'L?0.108\r\n']
ans is L?0.108

30.108
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n', b'L?-0.250\r\n']
ans is L?-0.250

29.75
M0L?

[b'M0L?\r\n

KeyboardInterrupt: 

In [190]:
ctl._serial.read_all()

b'OK\r\nM0L?\r\n-0.250\r\nPause:\r\n'

In [234]:
z.max_z=60
z.set_z(59.75)

0 59.75 60
M0L+29.750

[b'M0L+29.750\r\n', b'OK\r\n']


In [111]:
z.z_inversion=1

In [78]:
z.set_rel_z(-5)
z.read_z()

ValueError: could not convert string to float: 'Pause:'

In [258]:
# Check Homing Capability
z.homing()
print("Thorlabs should be at the 0 mm height, powerstep at 29.75 mm")

0 59.75 60
M0L+29.750

[b'M0L+29.750\r\n', b'OK\r\n']
Thorlabs should be at the 0 mm height, powerstep at 29.75 mm


In [264]:
# Check Movment capabilities 
z.set_z(29)
z.wait_for_move()
pos = z.read_z()
z.set_rel_z(0.05)
time.sleep(2)
final = z.read_z()

print(f"Initial position was {pos} mm, after move: {final} mm")
assert pos != final, print("Check Z Stage, it appears it did not move")

0 29 30
M0L-1.000

[b'M0L-1.000\r\n', b'OK\r\n']
M0L?

[b'M0L?\r\n', b'L?-13.693\r\n']
ans is L?-13.693

16.307000000000002
M0L?

[b'M0L?\r\n', b'L?-11.559\r\n']
ans is L?-11.559

18.441000000000003
M0L?

[b'M0L?\r\n', b'L?-9.424\r\n']
ans is L?-9.424

20.576
M0L?

[b'M0L?\r\n', b'L?-7.286\r\n']
ans is L?-7.286

22.714
M0L?

[b'M0L?\r\n', b'L?-5.148\r\n']
ans is L?-5.148

24.852
M0L?

[b'M0L?\r\n', b'L?-3.020\r\n']
ans is L?-3.020

26.98
M0L?

[b'M0L?\r\n', b'L?-1.247\r\n']
ans is L?-1.247

28.753
M0L?

[b'M0L?\r\n', b'L?-1.000\r\n']
ans is L?-1.000

29.0
M0L?

[b'M0L?\r\n', b'L?-1.000\r\n']
ans is L?-1.000

29.0
M0L?

[b'M0L?\r\n', b'L?-1.000\r\n']
ans is L?-1.000

29.0
M0L?

[b'M0L?\r\n', b'L?-1.000\r\n']
ans is L?-1.000

29.0
0 29.05 30
M0L-0.950

[b'M0L-0.950\r\n', b'OK\r\n']
M0L?

[b'M0L?\r\n', b'L?-0.950\r\n']
ans is L?-0.950

29.05
Initial position was 29.0 mm, after move: 29.05 mm


In [97]:
# Check Stop Capabilities
pos = z.read_z()
z.set_rel_z(5)
time.sleep(0.2)
z.stop()
z.read_z()

print(f"Thorlabs should stop moving before it reaches {pos+2}")
print(f"Final position: {z.read_z()}")

Thorlabs should stop moving before it reaches 31.743321
Final position: 29.747266


In [255]:
z.shutdown()
ctl.close()

In [98]:
z.shutdown()

Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveTimeoutException: Move failed to complete in time
   at Thorlabs.MotionControl.GenericMotorCLI.GenericMotorCLI.Wait(Int32 waitTimeout)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveJog(MotorDirection direction, Int32 waitTimeout)

Exception in thread Thread-33:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveTime

### Arduino PowerStep Motion